## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from anna import *
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = getData("../Weather_Database","WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aswan,EG,24.0934,32.9070,79.30,31,2,11.54,clear sky
1,1,Berlevag,NO,70.8578,29.0864,37.40,74,63,6.58,broken clouds
2,2,Puerto Lempira,HN,15.2667,-83.7667,76.84,89,76,7.81,broken clouds
3,3,Pedra Azul,BR,-16.0053,-41.2972,61.88,82,0,3.60,clear sky
4,4,Saint-Augustin,CA,51.2260,-58.6502,52.02,96,100,19.95,overcast clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
mintemp = input("Minimum Temperature:")
maxtemp = input("Maximum Temperature:")
mintemp,maxtemp

Minimum Temperature:60
Maximum Temperature:75


('60', '75')

In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
col = city_data_df['Max Temp']
theseTemps = city_data_df.loc[(col > int(mintemp)) & (col < int(maxtemp))]
theseTemps

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Pedra Azul,BR,-16.0053,-41.2972,61.88,82,0,3.60,clear sky
9,9,Bagotville,AU,-28.9833,153.4167,68.63,63,32,12.97,scattered clouds
11,11,Kerman,IR,30.2832,57.0788,74.70,6,0,1.50,clear sky
12,12,Rockland,US,41.1668,-74.0496,74.64,81,100,3.69,overcast clouds
17,17,Epe,NG,6.5841,3.9834,73.51,96,41,2.64,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
694,694,Araguari,BR,-18.6472,-48.1872,74.19,27,27,4.07,scattered clouds
695,695,Gagnoa,CI,6.1293,-5.9437,70.41,99,100,1.88,overcast clouds
714,714,Nova Vicosa,BR,-17.8919,-39.3719,73.83,87,4,10.36,clear sky
724,724,Greensburg,US,40.3015,-79.5389,66.61,97,75,0.00,mist


In [24]:
# 4a. Determine if there are any empty rows.
theseTemps.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
theseTemps = theseTemps.dropna()

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = theseTemps[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Pedra Azul,BR,61.88,clear sky,-16.0053,-41.2972,
9,Bagotville,AU,68.63,scattered clouds,-28.9833,153.4167,
11,Kerman,IR,74.70,clear sky,30.2832,57.0788,
12,Rockland,US,74.64,overcast clouds,41.1668,-74.0496,
17,Epe,NG,73.51,scattered clouds,6.5841,3.9834,
20,Los Llanos De Aridane,ES,71.35,clear sky,28.6585,-17.9182,
23,Police,PL,65.53,overcast clouds,53.5521,14.5718,
25,Bulaevo,KZ,64.45,overcast clouds,54.9056,70.4439,
26,Janauba,BR,74.77,clear sky,-15.8025,-43.3089,
27,Aktau,KZ,64.98,clear sky,43.6500,51.2000,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    name_address = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")      

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [40]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]

In [45]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
locations

,Lat,Lng
3,-16.0053,-41.2972
9,-28.9833,153.4167
11,30.2832,57.0788
12,41.1668,-74.0496
17,6.5841,3.9834
...,...,...
694,-18.6472,-48.1872
695,6.1293,-5.9437
714,-17.8919,-39.3719
724,40.3015,-79.5389


In [48]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))